# ai_vs_human_v1.4v.1 — Visualizer (Updated)

- Schema validation & helpful errors
- Output path safety
- Batch/collection mode via directory input
- Pinned plotting deps

*Input via `input()` is preserved. Chart definitions are isolated in a single cell.*

In [1]:
# CELL 1:
%pip install -q "pandas>=2.2.2" "numpy>=1.26.4" "matplotlib>=3.8.0" "seaborn>=0.13.0" "plotly>=5.22.0"

In [2]:
# CELL 2 — CHARTS & HELPERS (portable, PNG+SVG, no kaleido)

import os, json, glob
from pathlib import Path
import numpy as np
import pandas as pd

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns

ORIGIN_ORDER  = ["draft","refined","edited","none"]
ORIGIN_LABELS = {"draft":"Draft","refined":"Refined","edited":"Edited","none":"None"}
MOD_ORDER     = ["unchanged","minor","major"]
MOD_LABELS    = {"unchanged":"Unchanged","minor":"Minor","major":"Major"}

# ---------- DERIVATION HELPERS ----------
def _counts_from_dist(dist: dict, total: int, keys: list):
    vals = [dist.get(k, 0.0) for k in keys]
    # derive counts robustly (round then fix drift)
    raw = np.array(vals, dtype=float)
    pct = raw / raw.sum() if raw.sum() > 0 else np.zeros_like(raw)
    cnt = np.rint(pct * total).astype(int)
    # fix rounding drift
    diff = total - cnt.sum()
    if diff != 0:
        # distribute remainder to largest fractional parts
        frac = (pct * total) - np.floor(pct * total)
        order = np.argsort(-frac)
        for i in range(abs(diff)):
            idx = order[i % len(order)]
            cnt[idx] += 1 if diff > 0 else -1
    return list(cnt), list(pct)

def build_confusion(rows: list):
    mat = pd.DataFrame(0, index=ORIGIN_ORDER, columns=MOD_ORDER, dtype=int)
    for r in rows:
        o = (r.get("origin_version") or "none")
        m = r.get("modification_label")
        if o in ORIGIN_ORDER and m in MOD_ORDER:
            mat.loc[o, m] += 1
    return mat

# ---------- SAVE HELPERS ----------
def _ensure_dir(d):
    os.makedirs(d, exist_ok=True)

def _save_png_svg(fig, base):
    fig.savefig(base + ".png", dpi=220, bbox_inches="tight")
    fig.savefig(base + ".svg", bbox_inches="tight")
    plt.close(fig)

# ---------- PLOTS (Matplotlib) ----------
def plot_bar_with_labels(title, xticks, counts, percents, ylabel, base):
    fig, ax = plt.subplots(figsize=(8, 5))
    bars = ax.bar(xticks, counts)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_ylim(0, max(counts + [1]))  # at least 1 to show axes
    # add count + percent labels on bars
    for rect, c, p in zip(bars, counts, percents):
        ax.annotate(f"{c} ({p*100:.1f}%)",
                    xy=(rect.get_x() + rect.get_width()/2, rect.get_height()),
                    xytext=(0, 4), textcoords="offset points",
                    ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    _save_png_svg(fig, base)

def plot_confusion_heatmap(mat: pd.DataFrame, title: str, base: str):
    # percentage by row, plus counts in annotations
    counts = mat.values
    row_sums = counts.sum(axis=1, keepdims=True)
    perc = np.divide(counts, np.maximum(row_sums, 1), out=np.zeros_like(counts, dtype=float), where=row_sums>0)

    fig, ax = plt.subplots(figsize=(8.5, 6))
    sns.heatmap(perc, annot=False, cmap="Blues", vmin=0, vmax=1, cbar_kws={"format": '%.0f%%'}, ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Modification")
    ax.set_ylabel("Origin")
    ax.set_xticks(np.arange(len(MOD_ORDER)) + 0.5)
    ax.set_yticks(np.arange(len(ORIGIN_ORDER)) + 0.5)
    ax.set_xticklabels([MOD_LABELS[k] for k in MOD_ORDER], rotation=0)
    ax.set_yticklabels([ORIGIN_LABELS[k] for k in ORIGIN_ORDER], rotation=0)

    # overlay counts text
    for i in range(counts.shape[0]):
        for j in range(counts.shape[1]):
            ax.text(j + 0.5, i + 0.5, f"{counts[i, j]}\n({perc[i, j]*100:.0f}%)",
                    ha="center", va="center", color="black", fontsize=8)
    plt.tight_layout()
    _save_png_svg(fig, base)

# ---------- PUBLIC API ----------
def save_all_article_outputs(payload: dict, out_dir: str):
    """
    Saves:
      - origin_bar.(png|svg) with counts+%
      - mod_bar.(png|svg) with counts+%
      - origin×mod heatmap (png|svg)
      - summary CSV with totals & rates
    """
    _ensure_dir(out_dir)
    art  = payload["article_name"]
    summ = payload["summary"]
    rows = payload.get("rows", [])

    total = int(summ.get("counts", {}).get("sentences_final", len(rows) or 0))
    # if counts absent and rows empty, skip gracefully
    if total == 0:
        total = sum(int(round(v*1000)) for v in summ["origin_distribution"].values())  # arbitrary to avoid zero; charts will still work

    # Origin counts & perc
    origin_counts, origin_perc = _counts_from_dist(summ["origin_distribution"], total, ORIGIN_ORDER)
    plot_bar_with_labels(
        title=f"Origin Distribution — {art}",
        xticks=[ORIGIN_LABELS[k] for k in ORIGIN_ORDER],
        counts=origin_counts,
        percents=origin_perc,
        ylabel="Sentence Count",
        base=os.path.join(out_dir, f"{art}_origin_bar")
    )

    # Modification counts & perc
    mod_counts, mod_perc = _counts_from_dist(summ["modification_distribution"], total, MOD_ORDER)
    plot_bar_with_labels(
        title=f"Modification Distribution — {art}",
        xticks=[MOD_LABELS[k] for k in MOD_ORDER],
        counts=mod_counts,
        percents=mod_perc,
        ylabel="Sentence Count",
        base=os.path.join(out_dir, f"{art}_mod_bar")
    )

    # Origin × Modification heatmap (information equivalent to Sankey)
    mat = build_confusion(rows)
    if mat.values.sum() == 0 and total > 0:
        # If rows weren't provided, approximate by distributing modification by overall proportions
        approx = pd.DataFrame(0, index=ORIGIN_ORDER, columns=MOD_ORDER, dtype=int)
        for i, oc in enumerate(origin_counts):
            for j, mp in enumerate(mod_perc):
                approx.iloc[i, j] = int(round(oc * mp))
        mat = approx
    plot_confusion_heatmap(mat, f"Origin → Modification (Counts & % by Origin) — {art}",
                           os.path.join(out_dir, f"{art}_origin_mod_heatmap"))

    # Summary CSV
    summary_df = pd.DataFrame({
        "metric": (
            [f"origin_{k}" for k in ORIGIN_ORDER] +
            [f"mod_{k}" for k in MOD_ORDER] +
            ["total_sentences"]
        ),
        "count": origin_counts + mod_counts + [total],
        "percent": origin_perc + mod_perc + [1.0]
    })
    csv_path = os.path.join(out_dir, f"{art}_summary.csv")
    summary_df.to_csv(csv_path, index=False, encoding="utf-8")
    return [
        os.path.join(out_dir, f"{art}_origin_bar.png"),
        os.path.join(out_dir, f"{art}_origin_bar.svg"),
        os.path.join(out_dir, f"{art}_mod_bar.png"),
        os.path.join(out_dir, f"{art}_mod_bar.svg"),
        os.path.join(out_dir, f"{art}_origin_mod_heatmap.png"),
        os.path.join(out_dir, f"{art}_origin_mod_heatmap.svg"),
        csv_path
    ]

def save_all_collection_outputs(payloads: list, out_dir: str):
    _ensure_dir(out_dir)
    # Build per-article summary frame
    rows = []
    for p in payloads:
        art = p["article_name"]
        cnt = int(p["summary"].get("counts", {}).get("sentences_final", 0))
        o = p["summary"]["origin_distribution"]
        m = p["summary"]["modification_distribution"]
        row = {"article_name": art, "sentences_final": cnt}
        for k in ORIGIN_ORDER: row[f"origin_{k}"] = o.get(k, 0.0)
        for k in MOD_ORDER:   row[f"mod_{k}"]    = m.get(k, 0.0)
        rows.append(row)
    df = pd.DataFrame(rows).sort_values("article_name")
    csv_path = os.path.join(out_dir, "_collection_summary.csv")
    df.to_csv(csv_path, index=False, encoding="utf-8")

    # Simple collection bar charts (counts via distribution×count)
    def _bars(prefix, keys, labels_map, base):
        counts = []
        labels = df["article_name"].tolist()
        for _, r in df.iterrows():
            total = int(r["sentences_final"])
            vals = [r[f"{prefix}_{k}"] for k in keys]
            cts, _ = _counts_from_dist(dict(zip(keys, vals)), total, keys)
            counts.append(cts)
        counts = np.array(counts)  # shape: [articles, categories]
        # stacked bars by category
        fig, ax = plt.subplots(figsize=(max(8, 1.2*len(labels)), 5))
        bottom = np.zeros(len(labels))
        for j, k in enumerate(keys):
            ax.bar(labels, counts[:, j], bottom=bottom, label=labels_map[k])
            bottom += counts[:, j]
        ax.set_title(f"Collection — {prefix.capitalize()} (Counts)")
        ax.set_ylabel("Sentence Count")
        ax.legend()
        plt.xticks(rotation=20, ha="right")
        plt.tight_layout()
        _save_png_svg(fig, base)

    _bars("origin", ORIGIN_ORDER, ORIGIN_LABELS, os.path.join(out_dir, "_collection_origin_counts"))
    _bars("mod",    MOD_ORDER,   MOD_LABELS,    os.path.join(out_dir, "_collection_mod_counts"))
    return [
        os.path.join(out_dir, "_collection_summary.csv"),
        os.path.join(out_dir, "_collection_origin_counts.png"),
        os.path.join(out_dir, "_collection_origin_counts.svg"),
        os.path.join(out_dir, "_collection_mod_counts.png"),
        os.path.join(out_dir, "_collection_mod_counts.svg"),
    ]

# ---------- VALIDATION / DISCOVERY ----------
REQUIRED_TOP = ["article_name", "summary", "rows"]
REQUIRED_SUM = ["origin_distribution", "modification_distribution"]

def validate_payload(payload: dict, src: str) -> None:
    missing_top = [k for k in REQUIRED_TOP if k not in payload]
    if missing_top:
        raise KeyError(f"{src}: Missing required top-level key(s): {missing_top}")
    summary = payload.get("summary", {})
    missing_sum = [k for k in REQUIRED_SUM if k not in summary]
    if missing_sum:
        raise KeyError(f"{src}: Missing required summary key(s): {missing_sum}")

def load_one(json_path: str) -> dict:
    with open(json_path, "r", encoding="utf-8") as f:
        payload = json.load(f)
    validate_payload(payload, json_path)
    return payload

def discover_mode(path: str):
    if os.path.isdir(path):
        files = sorted(glob.glob(os.path.join(path, "*_complete_summary.json")))
        if not files:
            raise FileNotFoundError(f"No *_complete_summary.json files found in folder: {path}")
        return "dir", files
    if os.path.isfile(path):
        return "file", [path]
    raise FileNotFoundError(f"Path not found: {path}")

def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

In [3]:
# CELL 3 — RUNNER (PNG/SVG only; info-complete)

in_path = input("📁 Enter path to a *_complete_summary.json file OR a directory containing them: ").strip()

mode, files = discover_mode(in_path)
payloads = []
for f in files:
    try:
        payloads.append(load_one(f))
    except Exception as e:
        print(f"Skipping {f}: {e}")

if not payloads:
    raise SystemExit("No valid payloads to visualize.")

base_dir = os.path.dirname(files[0]) if mode == "file" else in_path
output_directory = os.path.join(base_dir, "viz_output")
ensure_dir(output_directory)
print(f"Output directory: {output_directory}")

# Per-article outputs
for p in payloads:
    for pth in save_all_article_outputs(p, output_directory):
        print(f"[Saved] {pth}")

# Collection outputs (if multiple inputs)
if len(payloads) > 1:
    for pth in save_all_collection_outputs(payloads, output_directory):
        print(f"[Saved] {pth}")

print("Visualization complete.")

📁 Enter path to a *_complete_summary.json file OR a directory containing them: /content/data_complete_summary.json
Output directory: /content/viz_output
[Saved] /content/viz_output/data_origin_bar.png
[Saved] /content/viz_output/data_origin_bar.svg
[Saved] /content/viz_output/data_mod_bar.png
[Saved] /content/viz_output/data_mod_bar.svg
[Saved] /content/viz_output/data_origin_mod_heatmap.png
[Saved] /content/viz_output/data_origin_mod_heatmap.svg
[Saved] /content/viz_output/data_summary.csv
Visualization complete.
